### Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [ ]:
!pip install konlpy

In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from konlpy.tag import Hannanum

In [5]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [7]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

In [38]:
from pandas.core.groupby.base import OutputKey
from torch import nn

### Data Load

In [8]:
cd /content/drive/MyDrive/multicampus/team_project1

/content/drive/MyDrive/multicampus/team_project1


In [13]:
train = pd.read_csv('data/train.csv', index_col=0)
test = pd.read_csv('data/test.csv', index_col=0)

### Data Preprocessing

In [14]:
train.head()

,sentiment1,sentiment2,dialog
0,기쁨,신이 난,지금 난 기분이 너무 좋아.
1,기쁨,신이 난,나도 조카가 생겨! 너무 기뻐.
2,불안,불안,거래처와의 다음 계약이 무산될까봐 불안해.
3,슬픔,염세적인,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...
4,슬픔,마비된,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.


In [16]:
train_senti1 = train[['sentiment1', 'dialog']]
test_senti1 = test[['sentiment1', 'dialog']]

In [17]:
tokenizer = get_tokenizer(Hannanum().morphs)

train_iter = iter(train_senti1.to_numpy())
test_iter = iter(test_senti1.to_numpy())

In [18]:
def yield_tokens(data_iter):
  for _, text in data_iter:
    yield tokenizer(text)

In [19]:
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [22]:
vocab(['지금', '난', '기분이', '너무', '좋아'])

[134, 4811, 0, 19, 0]

In [23]:
tokenizer('지금 난 기분이 너무 좋아.')

['지금', '나', 'ㄴ', '기분', '이', '너무', '좋', '아', '.']

In [24]:
text_transform = lambda x: vocab(tokenizer(x))
label_transform = lambda x: 1 if x=='기쁨' else 2 if x=='당황' else 3 if x=='상처' else 4 if x=='불안' else 5 if x=='슬픔' else 0

In [26]:
text_transform('지금 난 기분이 너무 좋아.')

[134, 6, 12, 97, 2, 19, 34, 7, 1]

In [27]:
print(label_transform('기쁨'))
print(label_transform('당황'))
print(label_transform('상처'))
print(label_transform('불안'))
print(label_transform('슬픔'))
print(label_transform('분노'))

1
2
3
4
5
0


- 데이터 batch : text -> list -> tensor -> `nn.EmbeddingBag`<br>(`nn.EmbeddingBag` 입력 위해 하나의 텐서로 결합)
- offset : text tensor에서 개별 시퀀스 시작 인덱스를 표현하기 위한 구분자(delimiter) tensor
- label : 개별 텍스트 항복의 레이블을 저장하는 tensor

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def collate_batch(batch):
  label_list, text_list, offsets = [], [], [0]

  for (_label, _text) in batch:
    label_list.append(label_transform(_label))
    processed_text = torch.tensor(text_transform(_text), dtype=torch.int64)
    text_list.append(processed_text)
    offsets.append(processed_text.size(0))

  label_list = torch.tensor(label_list, dtype=torch.int64)
  offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
  text_list = torch.cat(text_list)

  return label_list.to(device), text_list.to(device), offsets.to(device)

- `torch.utils.data.DataLoader`는 `getitem()`과 `len()` 프로토콜을 구현한 맵 형태(map-style)의 데이터셋으로 동작하며, 맵(map)처럼 인덱스/키로 데이터 샘플을 가져옴
- 셔플(shuffle) 인자를 `False`로 설정하면 순회 가능한(iterable) 데이터셋처럼 동작
- `collate_fn` 함수는 모델로 보내기 전 `DataLoader` 로부터 생성된 샘플 배치로 동작
- `collate_fn`의 입력은 `DataLoader`에 배치 크기(batch size)가 있는 배치(batch) 데이터
- collate_fn은 이를 미리 선언된 데이터 처리 파이프라인에 따라 처리

In [37]:
BATCH_SIZE = 100

train_iter = iter(train_senti1.to_numpy())
test_iter = iter(test_senti1.to_numpy())

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.95)
train_dataset, valid_dataset = random_split(train_dataset,
                                            [num_train, len(train_dataset)-num_train])

print('Train dataset: ', len(train_dataset))
print('Valid dataset: ', len(valid_dataset))
print('Test dataset: ', len(test_dataset))

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)

Train dataset:  86286
Valid dataset:  4542
Test dataset:  14025


### RNN model Define/Create

- 임베딩을 위한 `nn.EmbeddingBag` 레이어와 분류(classification) 목적을 위한 선형 레이어 `nn.Linear`로 구성
- 기본 모드가 "평균(mean)"인 ``nn.EmbeddingBag`` 은 임베딩들의 "가방(bag)"의 평균 값을 계산
- 텍스트(text) 항목들은 각기 그 길이가 다를 수 있지만, ``nn.EmbeddingBag`` 모듈은 텍스트의 길이를
오프셋(offset)으로 저장하고 있으므로 패딩(padding)이 필요하지 않음
- ``nn.EmbeddingBag``은 임베딩의 평균을 즉시 계산하기 때문에 
tensor들의 시퀀스를 처리할 때 성능 및 메모리 효율성 측면에서의 장점

In [39]:
class RNNModel(nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
    super(RNNModel, self).__init__()
    self.embed = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
    self.rnn = nn.RNN(embed_dim, hidden_dim)
    self.fc = nn.Linear(hidden_dim, num_class)

  def forward(self, input, offsets):
    output = self.embed(input, offsets)
    output,hidden = self.rnn(output)
    output = self.fc(output)
    return output

In [53]:
model = RNNModel(vocab_size = len(vocab),
                 embed_dim = 300,
                 hidden_dim = 100,
                 num_class = 6)  # 감정 대분류 6가지
model.to(device)

RNNModel(
  (embed): EmbeddingBag(23970, 300, mode=mean)
  (rnn): RNN(300, 100)
  (fc): Linear(in_features=100, out_features=6, bias=True)
)

### Train / Predict

In [54]:
def _train(dataloader):
  model.train()
  total_acc, total_count = 0, 0

  for idx, (label, text, offsets) in enumerate(dataloader):
    optimizer.zero_grad()
    predicted_label = model(text, offsets)
    loss = criterion(predicted_label, label)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
    optimizer.step()
    total_acc += (predicted_label.argmax(1) == label).sum().item()
    total_count += label.size(0)

  return total_acc / total_count

In [55]:
def _evaluate(dataloader):
  model.eval()
  total_acc, total_count = 0, 0

  with torch.no_grad():
    for idx, (label, text, offsets) in enumerate(dataloader):
      predicted_label = model(text, offsets)
      loss = criterion(predicted_label, label)
      total_acc += (predicted_label.argmax(1) == label).sum().item()
      total_count += label.size(0)

  return total_acc / total_count

In [56]:
import time

EPOCHS = 5  # epoch 수
LR = 5  # Learning Rate

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_acc = None

for epoch in range(1, EPOCHS+1):
  epoch_start_time = time.time()
  train_acc = _train(train_dataloader)
  valid_acc = _evaluate(valid_dataloader)

  if total_acc is not None and total_acc > valid_acc:
    scheduler.step()
  else:
    total_acc = valid_acc

  print('Epoch: {:2d}'.format(epoch),
        '| Elapsed Time: {:5.2f}s'.format(time.time() - epoch_start_time),
        '| Train Accuracy: {:.4f}'.format(train_acc),
        '| Valid Accuracy: {:.4f}'.format(valid_acc))

Epoch:  1 | Elapsed Time: 224.08s | Train Accuracy: 0.3920 | Valid Accuracy: 0.4432
Epoch:  2 | Elapsed Time: 216.09s | Train Accuracy: 0.4428 | Valid Accuracy: 0.4661
Epoch:  3 | Elapsed Time: 216.08s | Train Accuracy: 0.4638 | Valid Accuracy: 0.4758
Epoch:  4 | Elapsed Time: 217.50s | Train Accuracy: 0.4800 | Valid Accuracy: 0.5057
Epoch:  5 | Elapsed Time: 218.35s | Train Accuracy: 0.4902 | Valid Accuracy: 0.4996


평가 데이터셋(test set)을 통해 결과

In [ ]:
test_acc = _evaluate(test_dataloader)
print('Test Accuracy: {:.4f}'.format(test_acc))